In [1]:
!pip install scrapy
!pip install azure-storage-blob

Looking in indexes: https://artifactory.csx.com/artifactory/api/pypi/pypi-all/simple
Looking in indexes: https://artifactory.csx.com/artifactory/api/pypi/pypi-all/simple


In [2]:
!scrapy startproject areadevelopment

New Scrapy project 'areadevelopment', using template directory 'C:\Users\I8924\investment-tracker\.venv\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\I8924\investment-tracker\notebooks\areadevelopment

You can start your first spider with:
    cd areadevelopment
    scrapy genspider example example.com


In [3]:
cd areadevelopment

c:\Users\I8924\investment-tracker\notebooks\areadevelopment


c:\Users\I8924\investment-tracker\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
from azure.storage.blob import BlobServiceClient, BlobClient
from pymongo import MongoClient

# Azure Blob Storage connection string and container name
connection_string = (
    "DefaultEndpointsProtocol=https;"
    "AccountName=stazureaiinv156839749104;"
    "AccountKey=T2n3z+VCcpBeAKSoN7LdGfMJw6MJvx53I5yNUA9WhbD/CQehK00Ud6XgqOcpSTfc12T5SEDETtwk+AStlBikAw==;"
    "EndpointSuffix=core.windows.net"
)
container_name = "newsscrapping"

# Azure Cosmos DB Mongo DB connection string and database/collection names
cosmos_connection_string = "mongodb://industrialtrackers:28icWvqWAAoz9iUPStMuLC0EKe1y3SPTVyqLvDoYke36Hafq4V8lnHfM4nonHqsE0tmDaxfWyoAAACDb8zMiWQ==@industrialtrackers.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@industrialtrackers@"
cosmos_db_name = "indutrialtracker-db"
cosmos_collection_name = "indutrial-collections"

# Fetch all blobs from the container
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs()

# Connect to the Azure Cosmos DB Mongo DB
cosmos_client = MongoClient(cosmos_connection_string)
cosmos_db = cosmos_client[cosmos_db_name]
cosmos_collection = cosmos_db[cosmos_collection_name]

# Process each blob
for blob in blob_list:
    # Fetch blob content and metadata
    blob_client = container_client.get_blob_client(blob.name)
    response = blob_client.download_blob()
    blob_content = response.content_as_text(encoding="utf-8")
    properties = blob_client.get_blob_properties()
    blob_metadata = properties.metadata

    # Create a dictionary with the title, link, and article text
    data = {
        'title': blob_metadata.get('title'),
        'link': blob_metadata.get('link'),
        'article_text': blob_content
    }

    # Insert the dictionary into the Azure Cosmos DB Mongo DB collection
    cosmos_collection.insert_one(data)

# Close the Azure Cosmos DB Mongo DB connection
cosmos_client.close()

print("Finished processing all blobs and inserting them into the Azure Cosmos DB Mongo DB collection.")


C:\Users\I8526\AppData\Local\Temp\ipykernel_9572\1752826955.py:25: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  cosmos_client = MongoClient(cosmos_connection_string)


Finished processing all blobs and inserting them into the Azure Cosmos DB Mongo DB collection.


In [1]:
import os
import json
from openai import AzureOpenAI
from azure.storage.blob import BlobServiceClient, BlobClient

# Set the OpenAI API endpoint and key
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://investment-tracker.openai.azure.com/"
os.environ["AZURE_OPENAI_KEY"] = "ad53165125544166898b9574687ec65d"
deployment = "gpt-4"

# Azure Blob Storage connection string and container name
connection_string = (
    "DefaultEndpointsProtocol=https;"
    "AccountName=stazureaiinv156839749104;"
    "AccountKey=hufrDiFQA9SU4ZOB98B1fwXF0Ry5RQsDMTWvvXo9quC/fUMxQnAkxu4h6rlBWau2Az/KzxorDHmg+ASt4rffjQ==;"
    "EndpointSuffix=core.windows.net"
)
container_name = "newsscrapping"

# Fetch all blobs from the container
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs()

# Initialize an empty list to store the JSON objects
json_objects = []

# Process each blob
for blob in blob_list:
    # Fetch blob content and metadata
    blob_client = container_client.get_blob_client(blob.name)
    response = blob_client.download_blob()
    blob_content = response.content_as_text(encoding="utf-8")
    properties = blob_client.get_blob_properties()
    blob_metadata = properties.metadata

        # Use the Azure OpenAI service to extract the required entities from the article text
    messages = [
    {"role": "system", "content": "You are a helpful assistant that can extract information from a given text using the provided questions. Your task is to convert the extracted entities into a JSON-like dictionary format. Make sure to include strictly very small context in your answers and format the JSON strictly as per the example provided."},
    {"role": "user", "content": f"Please read the following text and answer the questions below to extract the necessary information:\n{blob_content}\n\n" +
                                      f"Question 1: What is the name of the company mentioned in the text, and provide strictly very small context about it?\n\n" +
                                      f"Question 2: What are the locational references mentioned in the text, such as address, lat/long, industrial park or site name, city/state, etc., and provide strictly very small context about them?\n\n" +
                                      f"Question 3: Is the company building a new facility or just acquiring land at the location mentioned in the text, and provide strictly very small context about it?\n\n" +
                                      f"Question 4: If the company is building a new facility, how much acreage is involved, and provide strictly very small context about it?\n\n" +
                                      f"Question 5: If the company is building a new facility, what is the square footage of the building, and provide strictly very small context about it?\n\n" +
                                      f"Question 6: What type of operation is the company engaged in at the location mentioned in the text, and provide strictly very small context about it?\n\n" +
                                      f"Question 7: What products or services does the company offer at the location mentioned in the text, and provide strictly very small context about them?\n\n" +
                                      f"Question 8: What is the capital investment involved in the project at the location mentioned in the text, and provide strictly very small context about it? Mention the $ symbol and use millions, billions, etc. as appropriate.\n\n" +
                                      f"Question 9: How many jobs will be created as a result of the project at the location mentioned in the text, and provide strictly very small context about it?\n\n" +
                                      f"Question 10: Who are the involved parties or partners, such as power provider, county officials, economic developers, railroad, in the project at the location mentioned in the text, and provide strictly very small context about them?\n\n" +
                                      f"Please provide the extracted information strictly in a JSON-like formats for reference one example given below\n\n" +
                                      f"{{\n" +
                                      f"    \"company_name\": \"Answer of company_name with strictly very small context\",\n" +
                                      f"    \"locational_references\": \"Answer of locational_references with strictly very small context\",\n" +
                                      f"    \"building_or_land\": \"Answer of building_or_land with strictly very small context\",\n" +
                                      f"    \"acreage\": \"Answer of acreage with strictly very small context\",\n" +
                                      f"    \"square_footage\": \"Answer of square_footage with strictly very small context\",\n" +
                                      f"    \"operation_type\": \"Answer of operation_type with strictly very small context\",\n" +
                                      f"    \"products_or_services\": \"Answer of products_or_services with strictly very small context\",\n" +
                                      f"    \"Engineering facility\": \"Answer of products_or_services with strictly very small context\",\n" +
                                      f"    \"capital_investment\": \"Answer of capital_investment with strictly very small context also mention $ symbol and mention millions, billions others\",\n" +
                                      f"    \"jobs_created\": \"Answer of jobs_created with strictly very small context\",\n" +
                                      f"    \"involved_parties\": \"Answer of involved_parties with strictly very small context\"\n" +
                                      f"}}"
                                      }
                       ]

    
    client = AzureOpenAI(
        api_key="ad53165125544166898b9574687ec65d",  
        api_version="2024-02-01",
        azure_endpoint ="https://investment-tracker.openai.azure.com/"
        )
    
    completion  = client.chat.completions.create(model="gpt-4", messages=messages, max_tokens=3000)
    
    # Extract the required entities from the Azure OpenAI response
    response= completion.choices[0].message.content.strip()
    
    data_dict = json.loads(response)
    data_dict["title"] = blob_metadata.get("title") if blob_metadata.get("title") else ""
    data_dict["link"] = blob_metadata.get("link") if blob_metadata.get("link") else ""
    data_dict["article_text"] = blob_content if blob_content else ""
    print(data_dict)

        # Convert the dictionary to a JSON object and add it to the list
    json_objects.append(data_dict)

# Save the list of JSON objects to a local JSON file
with open("C:\\Users\\I8526\\OneDrive - CSX\\Documents\\chatbot\\json_objects.json", "w") as f:
    json.dump(json_objects, f)

# Print the list of JSON objects
# print(json_objects)


In [111]:
with open("C:\\Users\\I8526\\OneDrive - CSX\\Documents\\chatbot\\json_objects_new.json", "w") as f:
    json.dump(json_objects, f)

In [3]:
# json_objects

In [31]:
%pwd

'C:\\Users\\I8526\\OneDrive - CSX\\Documents\\chatbot'

In [53]:
import json
import nltk
from nltk.tokenize import word_tokenize

# Download the NLTK punkt tokenizer
nltk.download("punkt")

# Load the JSON list from a local file
with open("json_objects.json", "r") as f:
    json_list = json.load(f)

# Initialize an empty list to store the stories and the final JSON objects
stories = []
final_json_objects = []

# Iterate through the JSON list in steps of 4
for i in range(0, len(json_list), 6):
    # Select the next 4 JSON objects from the list
    selected_json_list = json_list[i:i+6]

    # Iterate through the selected JSON list and create a story for each item
    for json_obj in selected_json_list:
        story = ""
        story += f"**Title:** {json_obj.get('title', '')}\n\n"
        story += f"**Company Name:** {json_obj.get('company_name', '')}\n\n"
        story += f"**Locational References:** {json_obj.get('locational_references', '')}\n\n"
        story += f"**Operation Type:** {json_obj.get('operation_type', '')}\n\n"
        story += f"**Products or Services:** {json_obj.get('products_or_services', '')}\n\n"
        if json_obj.get("building_or_land") and "building" in json_obj.get("building_or_land").lower():
            story += f"**Building or Land:** Building a new facility with a square footage of {json_obj.get('square_footage', '')}\n\n"
        elif json_obj.get("building_or_land") and "land" in json_obj.get("building_or_land").lower():
            story += f"**Building or Land:** Acquiring land with an acreage of {json_obj.get('acreage', '')}\n\n"
        else:
            story += f"**Building or Land:** {json_obj.get('building_or_land', '')}\n\n"
        story += f"**Capital Investment:** {json_obj.get('capital_investment', '')}\n\n"
        story += f"**Jobs Created:** {json_obj.get('jobs_created', '')}\n\n"
        story += f"**Involved Parties:** {json_obj.get('involved_parties', '')}\n\n"
        story += f"**Source Link:** {json_obj.get('link', '')}"

        # Add the story to the list
        stories.append(story)

    # If there are stories in the list, merge them into one and create a JSON object
    if stories:
        merged_stories = "\n\n---\n\n".join(stories)
        json_object = {"content": merged_stories}
        final_json_objects.append(json_object)
        # Reset the stories list
        stories = []

# If there are less than 4 modified JSON objects, merge them into one
if stories:
    merged_stories = "\n\n---\n\n".join(stories)
    json_object = {"content": merged_stories}
    final_json_objects.append(json_object)

# Save the final JSON objects to a local file
with open("merged_json_objects.json", "w") as f:
    json.dump(final_json_objects, f, indent=4)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\I8526\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import os
import json
from openai import AzureOpenAI

# Initialize OpenAI API client
client = AzureOpenAI(
    api_key="ad53165125544166898b9574687ec65d",  
    api_version="2024-02-01",
    azure_endpoint ="https://investment-tracker.openai.azure.com/"
)

def get_embeddings(text, model='text-embedding-3-large'):
    return client.embeddings.create(input=text, model=model).data[0].embedding

def process_json(json_path, output_dir):
    # Read JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Initialize output list
    output_list = []

    # Iterate over items in JSON list
    for i, item in enumerate(data):
        d = {}
        d['id'] = str(i)
        d['content'] = str(item['content'])
        d['embedding_content'] = get_embeddings(item['content'])
        output_list.append(d)

    # Save output list to JSON file in specified directory
    output_file = os.path.join(output_dir, f"{os.path.basename(json_path)[:-5]}_embedded.json")
    with open(output_file, "w") as f:
        json.dump(output_list, f)


In [57]:
process_json('merged_json_objects.json', 'embedded.json')

In [15]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient, SearchIndexingBufferedSender
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,
)
from azure.search.documents.indexes.models import (
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SearchField,
    SemanticSearch,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SearchField,
    VectorSearch,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)

os.environ["AZURE_SEARCH_KEY"] = "pAVCNvVE4EHgqREV6Zdu6ejfiHHEwlj0a2PLJfiJ6mAzSeApVIZj"
# load_dotenv()

service_endpoint = "https://investmenttracker.search.windows.net"

credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))

def create_index(index_name):
    index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)

    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, filterable=True, searchable=True, facetable=True),
    SearchField(name="embedding_content", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=3072, vector_search_profile_name="myHnswProfile")
]


    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                kind=VectorSearchAlgorithmKind.HNSW,
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE
                )
            ),
            ExhaustiveKnnAlgorithmConfiguration(
                name="myExhaustiveKnn",
                kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
                parameters=ExhaustiveKnnParameters(
                    metric=VectorSearchAlgorithmMetric.COSINE
                )
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            ),
            VectorSearchProfile(
                name="myExhaustiveKnnProfile",
                algorithm_configuration_name="myExhaustiveKnn",
            )
        ]
    )

    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            content_fields=[SemanticField(field_name="content")],
            keywords_fields=[SemanticField(field_name="content")]
        )
    )

    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=SemanticSearch(configurations=[semantic_config]))

    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

def upload_documents(index_name, documents):
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
    result = search_client.upload_documents(documents)
    print(f"Uploaded {len(documents)} documents to {index_name}")

embedding_json_dir = 'embedded'

# for file in os.listdir(embedding_json_dir):
#     if file.endswith('.json'):
#         index_name = os.path.splitext(file)[0].lower().replace(" ", "-").strip("-")[:128]

#         print(f'Creating index for {index_name}')
#         create_index(index_name)

#         with open(os.path.join(embedding_json_dir, file), 'r') as f:
#             documents = json.load(f)

#         print(f'Uploading documents to {index_name}')
#         upload_documents(index_name, documents)


In [16]:
import os
import json
from openai import AzureOpenAI

In [24]:
import os
import json
from openai import AzureOpenAI
import azure.search.documents as search_client
from azure.core.credentials import AzureKeyCredential
from IPython.display import HTML

def generate_completion(response):
    messages = [
        {"role": "system", "content": "you are expert in HTML you are given response as a expert you convert response in HTML formats which is very fancy for user to visualize"},
        {"role": "user", "content": response}
    ]

    client = AzureOpenAI(
        api_key="ad53165125544166898b9574687ec65d",
        api_version="2024-02-01",
        azure_endpoint ="https://investment-tracker.openai.azure.com/"
    )

    completion  = client.chat.completions.create(model="gpt-4", messages=messages, max_tokens=3000)

    # Extract the required entities from the Azure OpenAI response
    html_string = completion.choices[0].message.content.strip()
    return html_string

In [34]:
def generate_answer(context):
    messages = [
        {"role": "system", "content": f"You are a highly skilled and knowledgeable AI assistant, capable of providing accurate and insightful answers to a wide range of questions. You have been given the following query: {query}. Your task is to use the provided context to derive an answer that is both relevant and informative. You must ensure that your answer is strictly based on the context, and that you do not make any assumptions or speculations. If the answer is not available in the context, you should respond with 'Information is not available'."},
        {"role": "user", "content": context}
           ]

    client = AzureOpenAI(
        api_key="ad53165125544166898b9574687ec65d",
        api_version="2024-02-01",
        azure_endpoint ="https://investment-tracker.openai.azure.com/"
    )

    completion  = client.chat.completions.create(model="gpt-4", messages=messages, max_tokens=3000)

    # Extract the required entities from the Azure OpenAI response
    ans = completion.choices[0].message.content.strip()
    return ans


In [61]:
query = '''list all projects worth more than 1 billion dollor tell about their geospatial location that should create more than 300 jobs 
provide nature of investment and parties between investment give source of information'''

In [62]:
query_vector = get_embeddings(query, model='text-embedding-3-large')

In [63]:
index_name = 'embedded'

In [64]:
search_client = SearchClient(service_endpoint, index_name, credential)  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding_content")

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["content"],
    top=6
)  

result_content = ""
for result in results:
    result_content += ',' + result['content']



In [65]:
ans = generate_answer(result_content)
# response = generate_completion(ans)
# display(HTML(response))
ans

"Based on the information provided, here is a summary of projects worth more than 1 billion dollars, their geospatial location, nature of investment, job creation forecast, involved parties, and the source of information:\n\n### Project Title: Microsoft Establishes Mount Pleasant, Wisconsin, Data Center Operations\n\n- **Geospatial Location:** Mount Pleasant, Wisconsin, reference to UW-Milwaukee.\n- **Nature of Investment:** Data center operations including a cloud and AI infrastructure, an AI co-innovation lab, and an AI skilling initiative.\n- **Capital Investment:** $3.3 billion by 2026.\n- **Jobs Created:** Information on the number of jobs created is not provided.\n- **Involved Parties:** Wisconsin Economic Development Corporation (WEDC), Racine County, the village of Mount Pleasant, and the Milwaukee 7 regional economic development organization.\n- **Source of Information:** [Area Development Link](https://www.areadevelopment.com/newsItems/5-16-2024/microsoft-mount-pleasant-wisco

In [59]:
display(HTML(ans))

In [47]:
search_client = SearchClient(service_endpoint, index_name, credential)  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding_content")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["content"],
    top=6
)  

result_content = ""
for result in results:
    result_content += ',' + result['content']

 

In [48]:
# ans = generate_answer(result_content)
# response = generate_completion(ans)
# display(HTML(response))

In [60]:
ans = generate_answer(result_content)
# response = generate_completion(ans)
# display(HTML(response))
ans

"Based on the provided context, here are projects worth more than 1 billion dollars that are expected to create more than 300 jobs, along with their geospatial location, nature of investment, and the parties involved:\n\n**Title:** Microsoft Establishes Mount Pleasant, Wisconsin, Data Center Operations\n\n- **Geospatial Location:** Mount Pleasant, Wisconsin\n- **Nature of Investment:** Data center operations with a focus on cloud and AI infrastructure, AI co-innovation lab, and AI skilling initiative.\n- **Capital Investment:** $3.3 billion by 2026.\n- **Jobs Created:** Information not available.\n- **Involved Parties:** The Wisconsin Economic Development Corporation (WEDC), Racine County, the village of Mount Pleasant, and the Milwaukee 7 regional economic development organization.\n\n**Source Link:** [Microsoft Mount Pleasant, Wisconsin](https://www.areadevelopment.com/newsItems/5-16-2024/microsoft-mount-pleasant-wisconsin.shtml)\n\n**Title:** Rivian Automotive Expands Normal, Illino

In [50]:
search_client = SearchClient(service_endpoint, index_name, credential)  
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, 
                               fields="embedding_content",exhaustive=True)

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["content"],
    top=1
)  
  


result_content = ""
for result in results:
    result_content += ',' + result['content']
 

In [ ]:
# ans = generate_answer(result_content)
# response = generate_completion(ans)
# response

In [51]:
ans = generate_answer(result_content)
# response = generate_completion(ans)
# display(HTML(response))
ans

'Information is not available.'

In [52]:
search_client = SearchClient(service_endpoint, index_name, credential)  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding_content")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["content"],
    top=1
)  


result_content = ""
for result in results:
    result_content += ',' + result['content']

In [ ]:
# ans = generate_answer(result_content)
# response = generate_completion(ans)
# response

In [53]:
search_client = SearchClient(service_endpoint, index_name, credential)  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding_content")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["content"],
    top=6
)  

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["content"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='my-semantic-config', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)  
 

In [51]:
semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

In [52]:
for result in results:
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}\n")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Reranker Score: 1.7550550699234009


KeyError: 'line'